In [1]:
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.io_utils import HDF5Matrix
from keras import metrics
import numpy as np

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [3]:
import pandas as pd

In [4]:
data_polls='train2016.csv'
submission = 'test2016.csv'

In [5]:
DataFrame = pd.read_csv(data_polls)
DataFrame_sub = pd.read_csv(submission)
DataFrame.head()

,USER_ID,YOB,Gender,Income,HouseholdStatus,EducationLevel,Party,Q124742,Q124122,Q123464,...,Q100010,Q99716,Q99581,Q99480,Q98869,Q98578,Q98059,Q98078,Q98197,Q96024
0,1,1938.0,Male,NaN,Married (w/kids),NaN,Democrat,No,NaN,No,...,Yes,No,No,NaN,No,NaN,Only-child,No,No,Yes
1,4,1970.0,Female,over $150000,Domestic Partners (w/kids),Bachelor's Degree,Democrat,NaN,Yes,No,...,NaN,NaN,NaN,No,No,No,Only-child,Yes,No,No
2,5,1997.0,Male,$75000 - $100000,Single (no kids),High School Diploma,Republican,NaN,Yes,Yes,...,Yes,No,No,No,Yes,No,Yes,No,Yes,No
3,8,1983.0,Male,$100001 - $150000,Married (w/kids),Bachelor's Degree,Democrat,No,Yes,No,...,No,No,No,Yes,Yes,No,Yes,No,No,Yes
4,9,1984.0,Female,$50000 - $74999,Married (w/kids),High School Diploma,Republican,No,Yes,No,...,Yes,No,No,Yes,No,No,Yes,No,No,Yes


In [6]:
#training data after removing the user id and lalbel columns
features_train_pd = DataFrame.ix[:,[2,3,4,5]+range(7,108)]
labels_train_pd = DataFrame.ix[:,6]


# Only for the submission data
sub_features_train_pd = DataFrame_sub.ix[:,[2,3,4,5]+range(6,107)]
sub_userid = DataFrame_sub.ix[:,[0]]


In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in list(features_train_pd):
    try:
        features_train_pd[i] = le.fit_transform(features_train_pd[i])
    except:
        print("Could not convert ", i)
        
        
labels_train_pd = le.fit_transform(labels_train_pd)

C:\Users\User\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
C:\Users\User\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#converting into numpy arrays to feed into the model

features = features_train_pd.as_matrix()
labels = labels_train_pd
#sub_features = sub_features_train_pd.as_matrix()

In [9]:
#train-test split

from sklearn.cross_validation import train_test_split

features_train,features_test,labels_train,labels_test = train_test_split(features,labels)

In [10]:
#Defining the model

model = Sequential()
model.add(Dense(64, input_shape=(105,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(features_train, labels_train, batch_size=32, shuffle='batch')


Epoch 1/10
4176/4176 [==============================] - 83s - loss: 0.7126    
Epoch 2/10
4176/4176 [==============================] - 89s - loss: 0.6678    
Epoch 3/10
4176/4176 [==============================] - 85s - loss: 0.6588    
Epoch 4/10
4176/4176 [==============================] - 88s - loss: 0.6464    
Epoch 5/10
4176/4176 [==============================] - 82s - loss: 0.6398    
Epoch 6/10
4176/4176 [==============================] - 90s - loss: 0.6310    
Epoch 7/10
4176/4176 [==============================] - 96s - loss: 0.6249    
Epoch 8/10
4176/4176 [==============================] - 93s - loss: 0.6127    
Epoch 9/10
4176/4176 [==============================] - 86s - loss: 0.6058    
Epoch 10/10
4176/4176 [==============================] - 99s - loss: 0.5978    


## Accuracy

In [11]:
model.evaluate(features_test, labels_test, batch_size=32)

1376/1392 [============================>.] - ETA: 0s

0.68386479218800866

## Submission

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in list(sub_features_train_pd):
    try:
        sub_features_train_pd[i] = le.fit_transform(sub_features_train_pd[i])
    except:
        print("Could not convert ", i)
        
        


C:\Users\User\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
sub_features = sub_features_train_pd.as_matrix()


In [30]:
probs = model.predict(sub_features)
predictions = [int(round(x)) for x in probs]
pred2=[]

for i in predictions:
    if i == 1:
        pred2.append('Republican')
    else:
        pred2.append('Democrat')
        
final_submission = np.c_[sub_userid,pred2]

output = pd.DataFrame(final_submission)

output.to_csv('out.csv')